In [ ]:
#Imports & Device
import torch
import torch.nn as nn
import torch.nn.functional as F
import segmentation_models_pytorch as smp

import numpy as np
import glob
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

import cv2
from io import BytesIO
import matplotlib.pyplot as plt

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

In [ ]:
#Dataset (Train + Val)
IMG_SIZE = 256
NUM_CLASSES = 5

class MapDataset(Dataset):
    def __init__(self, img_dir, lbl_dir):
        self.img_paths = sorted(glob.glob(img_dir + "/*.jpg"))
        self.lbl_paths = sorted(glob.glob(lbl_dir + "/*.txt"))

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img = Image.open(self.img_paths[idx]).convert("RGB")
        img = img.resize((IMG_SIZE, IMG_SIZE))
        arr = np.array(img) / 255.0
        arr = arr.transpose(2,0,1).astype("float32")
        label = int(open(self.lbl_paths[idx]).read().strip())
        return torch.tensor(arr), torch.tensor(label)

In [ ]:
TRAIN_DIR = "data/train"
VAL_DIR   = "data/val_1"

train_ds = MapDataset(f"{TRAIN_DIR}/images", f"{TRAIN_DIR}/labels")
val_ds   = MapDataset(f"{VAL_DIR}/images",   f"{VAL_DIR}/labels")

train_dl = DataLoader(train_ds, batch_size=8, shuffle=True)
val_dl   = DataLoader(val_ds, batch_size=1, shuffle=False)

print("Train samples:", len(train_ds))
print("Val samples:", len(val_ds))

In [ ]:
#Load base model
base_model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights=None,
    in_channels=3,
    classes=NUM_CLASSES
).to(DEVICE)

base_model.load_state_dict(
    torch.load("models/unet_resnet34.pth", map_location=DEVICE)
)
base_model.eval()

print("✅ Base model loaded")

In [ ]:
#FGSM Generator
def fgsm_generate(model, x, y, eps):
    x_adv = x.clone().detach().to(DEVICE)
    x_adv.requires_grad = True

    out = model(x_adv).mean(dim=(2,3))
    loss = F.cross_entropy(out, y)

    model.zero_grad()
    loss.backward()

    adv = x_adv + eps * x_adv.grad.sign()
    return torch.clamp(adv, 0, 1).detach()

In [ ]:
#FGSM Adversarial Fine-Tuning Loop
adv_model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights=None,
    in_channels=3,
    classes=NUM_CLASSES
).to(DEVICE)

adv_model.load_state_dict(
    torch.load("models/unet_resnet34.pth", map_location=DEVICE)
)

optimizer = torch.optim.Adam(adv_model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

EPOCHS_ADV = 3
EPS_TRAIN = 0.02

print("🛡️ Starting FGSM adversarial fine-tuning...\n")

for epoch in range(EPOCHS_ADV):
    adv_model.train()
    running_loss = 0.0

    for imgs, labels in tqdm(train_dl, desc=f"Adv Epoch {epoch+1}/{EPOCHS_ADV}"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        # Generate FGSM adversarial examples
        adv_imgs = fgsm_generate(adv_model, imgs, labels, EPS_TRAIN)

        optimizer.zero_grad()

        out_clean = adv_model(imgs).mean(dim=(2,3))
        out_adv   = adv_model(adv_imgs).mean(dim=(2,3))

        loss = 0.5 * criterion(out_clean, labels) + \
               0.5 * criterion(out_adv, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} | Avg Loss: {running_loss/len(train_dl):.4f}")

In [ ]:
#Save FGSM-Trained Model
torch.save(
    adv_model.state_dict(),
    "models/unet_resnet34_adv.pth"
)

print("✅ FGSM adversarially trained model saved")

In [ ]:
#FGSM Evaluation Function
def eval_fgsm(model, dataloader, eps):
    correct, total = 0, 0

    for x, y in dataloader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        adv = fgsm_generate(model, x, y, eps)

        with torch.no_grad():
            out = model(adv).mean(dim=(2,3))
            pred = out.argmax(dim=1)

        correct += (pred == y).sum().item()
        total += 1

    return 100 * correct / total

In [ ]:
#FGSM Results (Base vs Adv-Trained)
EPS_LIST = [0.0, 0.02, 0.04, 0.06, 0.08]

base_results = []
adv_results  = []

for eps in EPS_LIST:
    base_results.append(eval_fgsm(base_model, val_dl, eps))
    adv_results.append(eval_fgsm(adv_model,  val_dl, eps))
print("\nFGSM Robustness Comparison")
print("Epsilon | Base (%) | FGSM-Trained (%)")
print("-------------------------------------")

for i, eps in enumerate(EPS_LIST):
    print(f"{eps:6.2f} | {base_results[i]:8.2f} | {adv_results[i]:14.2f}")

In [ ]:
#PGD Attack Function
def pgd_attack(model, x, y, eps=0.04, alpha=0.005, steps=20):
    model.eval()

    x_orig = x.clone().detach().to(DEVICE)
    x_adv = x_orig.clone()

    for _ in range(steps):
        x_adv.requires_grad = True

        out = model(x_adv).mean(dim=(2,3))
        loss = F.cross_entropy(out, y)

        model.zero_grad()
        loss.backward()

        # PGD update
        x_adv = x_adv + alpha * x_adv.grad.sign()

        # Project back to epsilon-ball
        perturb = torch.clamp(x_adv - x_orig, -eps, eps)
        x_adv = torch.clamp(x_orig + perturb, 0, 1).detach()

    return x_adv

In [ ]:
#Hybrid Defense
def jpeg_def(arr, quality=40):
    im = Image.fromarray(arr)
    buf = BytesIO()
    im.save(buf, format="JPEG", quality=quality)
    buf.seek(0)
    return np.array(Image.open(buf).convert("RGB"))

def gaussian_def(arr, k=3):
    return cv2.GaussianBlur(arr, (k,k), 0)

def median_def(arr, k=3):
    return cv2.medianBlur(arr, k)

def bit_def(arr, bits=5):
    shift = 8 - bits
    return ((arr >> shift) << shift)

def hybrid_defense(arr):
    arr = jpeg_def(arr, quality=40)
    arr = gaussian_def(arr, k=3)
    arr = median_def(arr, k=3)
    arr = bit_def(arr, bits=5)
    return arr

def to_tensor(arr):
    return torch.tensor(arr/255.0, dtype=torch.float32).permute(2,0,1)

In [ ]:
#PGD Evaluation Function (With / Without Defense)
def eval_pgd(model, dataloader, eps, hybrid=False):
    correct, total = 0, 0

    for x, y in tqdm(dataloader, leave=False):
        x, y = x.to(DEVICE), y.to(DEVICE)

        adv = pgd_attack(model, x, y, eps=eps)

        if hybrid:
            arr = (adv[0].permute(1,2,0).cpu().numpy() * 255).astype(np.uint8)
            arr_def = hybrid_defense(arr)
            tens = to_tensor(arr_def).unsqueeze(0).to(DEVICE)

            with torch.no_grad():
                out = model(tens).mean(dim=(2,3))
        else:
            with torch.no_grad():
                out = model(adv).mean(dim=(2,3))

        pred = out.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += 1

    return 100 * correct / total

In [ ]:
#Run PGD Evaluation
EPS_LIST = [0.0, 0.02, 0.04, 0.06, 0.08]

pgd_no_def  = []
pgd_hybrid  = []

print("🔥 Running PGD evaluation...\n")

for eps in EPS_LIST:
    print(f"Evaluating eps = {eps}")
    pgd_no_def.append(eval_pgd(base_model, val_dl, eps, hybrid=False))
    pgd_hybrid.append(eval_pgd(base_model, val_dl, eps, hybrid=True))

In [ ]:
#PGD Results Table
print("\nPGD Robustness Comparison")
print("Epsilon | No Defense (%) | Hybrid Defense (%)")
print("---------------------------------------------")

for i, eps in enumerate(EPS_LIST):
    print(f"{eps:6.2f} | {pgd_no_def[i]:14.2f} | {pgd_hybrid[i]:17.2f}")

In [ ]:
#PGD Accuracy Plot
plt.figure(figsize=(7,5))
plt.plot(EPS_LIST, pgd_no_def, marker='o', label="PGD (No Defense)")
plt.plot(EPS_LIST, pgd_hybrid, marker='o', label="PGD (Hybrid Defense)")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy (%)")
plt.title("PGD Attack Robustness with Hybrid Defense")
plt.grid(True)
plt.legend()
plt.show()